# I. Tweet Data Cleaning



In [2]:
## Tweet Table

In [3]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time

In [4]:
#function to cheack whether 
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [5]:
%%time

#making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']

want_drop = input()
if(want_drop=='yes'): ## if type yes, then it will drop tweet table from mongodb to prepare new importing.
    db.tweet.drop()
    
tweet = db['tweet']

yes
CPU times: user 9.11 ms, sys: 3.83 ms, total: 12.9 ms
Wall time: 14.3 s


In [6]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [7]:
#yelin directory '/Users/yelin/Downloads/corona-out-3/corona-out-3'
#Mo 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-2'
# Hui Wang  file = '/Users/huiwang/Downloads/corona-out-2'
file = '/Users/huiwang/Downloads/corona-out-3'

In [8]:
%%time

#id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en'):
            
            #case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :

                Created_at = pd.to_datetime(data['retweeted_status']['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['retweeted_status']['user']['id']
                ID = data['retweeted_status']['id']
                retweet_count = data['retweeted_status']['retweet_count']
                favorite_count = data['retweeted_status']['favorite_count']
                reply_count = data['retweeted_status']['reply_count']

                #check whether there is extension version of that tweet or not        
                if (check_var(data['retweeted_status'], 'extended_tweet')) :
                    Text = data['retweeted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = data['retweeted_status']['text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

                #check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :

                    #check whether there is extension version of the original tweet or not 
                    if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                        for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                            hashtag.append(t['text'])                
                    else :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                        for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                            hashtag.append(t['text'])               

            #case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that quote tweet or not 
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text'] + ' || ' 
                else:
                    Text = data['text'] + ' || '

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

                #check whether there is extension version of the originl tweet or not 
                if (check_var(data['quoted_status'], 'extended_tweet')) :
                    Text = Text + data['quoted_status']['extended_tweet']['full_text']
                    for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else:
                    Text = Text + data['quoted_status']['text']
                    for t in data['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

            #if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that tweet or not  
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text']
                else :
                    Text = data['text']

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

            tw = {'id' : ID,
                  'user_id' : UserID,
                  'created_at' : Created_at,
                  'text' : Text,
                  'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'reply_count' : reply_count,
                  'hashtags': hashtag
                 }

            #if the data is already in our database, we update it 
            if (tweet.count_documents({'id' : tw['id']}) > 0) :
                db.tweet.update_one({'id' : tw['id']},
                                    {'$set' : {'retweet_count' : tw['retweet_count'],
                                               'favorite_count' : tw['favorite_count'],
                                               'reply_count' : tw['reply_count']}})
            #if it is not, we insert it    
            else :
                tweet.insert_one(tw)
            

tweet.create_index([ ('id', 1) ])

CPU times: user 1min 32s, sys: 5.04 s, total: 1min 37s
Wall time: 13min 38s


'id_1'

In [9]:
tweet_table = pd.DataFrame(tweet.find())
tweet_table.shape

(19076, 9)

In [43]:
tweet_table.head(5)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea5fc384d254083e4e01ccb,1253949413191344128,207809313,"April 25, 2020, 07:30:12 AM",India’s war with Corona is ongoing.\n\nPlay yo...,398,2144,108,[FeedTheNeedy]
1,5ea5fc384d254083e4e01ccc,1253992905703862272,858859031464751104,"April 25, 2020, 10:23:01 AM",VIDEO | 25.04.20\n\n&gt; Monsta X appears on Y...,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]"
2,5ea5fc384d254083e4e01ccd,1253697983343816706,2719753171,"April 24, 2020, 02:51:06 PM","Sri ByReddy Rajasekhar Reddy garu,a leader,who...",2238,7243,47,[]
3,5ea5fc384d254083e4e01cce,1254022773598572544,1091660129894838272,"April 25, 2020, 12:21:42 PM",@VinceMcMahon @TripleH We hereby honor to anno...,0,0,0,[]
4,5ea5fc384d254083e4e01ccf,1254022776094105602,375777294,"April 25, 2020, 12:21:43 PM",im making 17 a hr doing nothing 😭😭😭😭 i love th...,0,0,0,[]






# II. User Data Cleaning





In [10]:
## User Table

In [11]:
import pandas as pd
import psycopg2
import json

In [12]:
##Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()

want_drop = input()
if(want_drop=='yes'): ## if type yes, then it will drop database from postgreSQL to prepare new importing.
    cur.execute("DROP DATABASE IF EXISTS twitter_users" )
    
cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

yes


In [13]:
#Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id varchar(50) PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [14]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [15]:
%%time

#This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        if (data['lang'] == 'en'): # only 
            
            if (check_var(data, 'retweeted_status')):

                user_id = data['retweeted_status']['user']['id_str']
                name = data['retweeted_status']['user']['name']
                user_name = data['retweeted_status']['user']['screen_name']
                verified_status = data['retweeted_status']['user']['verified']
                followers_count = data['retweeted_status']['user']['followers_count']
                friends_count = data['retweeted_status']['user']['friends_count']
                statuses_count = data['retweeted_status']['user']['statuses_count']
                user_location = data['retweeted_status']['user']['location']
                favourites_count= data['retweeted_status']['user']['favourites_count']


            else :
                user_id = data['user']['id_str']
                name = data['user']['name']
                user_name = data['user']['screen_name']
                verified_status = data['user']['verified']
                followers_count = data['user']['followers_count']
                friends_count = data['user']['friends_count']
                statuses_count = data['user']['statuses_count']
                user_location = data['user']['location']
                favourites_count= data['user']['favourites_count']


            #insert the data into the PostgreSQL database
            store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

CPU times: user 1min 15s, sys: 23.6 s, total: 1min 39s
Wall time: 58min 43s


In [17]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [18]:
user_table = create_pandas_table("SELECT * FROM users_data")
user_table.shape

(15673, 9)

In [44]:
user_table.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,375777294,TeéLaneeë🌺,TWD40_,False,637,408,18976,None,1325
1,1132273796138905600,Terri Kamp,RampTheresa,False,1522,256,20861,None,29166
2,301470336,BABA ALI,_FreshAA,False,9406,1870,264287,"Chicago, IL",857
3,923169415054680064,Quantum,QuantumAspect,False,3196,1001,15219,The Netherlands,119803
4,819200364675072000,Internet Historian,NetHistorian,False,259632,547,1087,None,4280






# III. Searching Application





In [19]:
## Install PySpark
#!curl -O https://d3kbcqa49mib13.cloudfront.net/spark-2.2.0-bin-hadoop2.7.tgz
#!tar -xvf spark-2.2.0-bin-hadoop2.7.tgz

In [20]:
# Configure your python environment by uncommenting the line below.
#!pip install findspark

In [21]:
## Launch PySpark
import os
import findspark
os.environ["PYSPARK_PYTHON"] = "python3"
findspark.init("spark-2.2.0-bin-hadoop2.7",)

Import components of SparkSQL

In [22]:
from pyspark.sql import SparkSession, Column, Row, functions as F

Initialize the SparkSQL session which contains a basic Spark Context. This may take a few moments to launch the cluster of (typically 4 to 8 python jobs in the background). Note that you are running Spark locally (.master).

In [23]:
spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("SparkLecture694Example")
        .getOrCreate()
)
sc = spark.sparkContext

In [24]:
# BUG tweet_table_spark = spark.createDataFrame(tweet_table) 

In [25]:
# BUG user_table_spark = spark.createDataFrame(user_table)

Implement 1

In [26]:
%%time

# using MongoDB
# most popular original tweet ranked by the retweet_count
pop_retweet = pd.DataFrame(tweet.find().sort("retweet_count",-1))
pop_retweet.head(5)

CPU times: user 305 ms, sys: 38.4 ms, total: 343 ms
Wall time: 677 ms


,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea5fccf4d254083e4e03e3f,1238264431320215553,1100261477989126145,"March 13, 2020, 12:43:40 AM",*corona virus enters my body*\n\nThe 4 Flintst...,237307,811062,1811,[]
1,5ea5fc9c4d254083e4e0378e,1240334979701395458,1112592502727548928,"March 18, 2020, 05:51:18 PM",When this Corona shit passes we have to promis...,181584,764405,1595,[]
2,5ea5fc604d254083e4e02c0c,1237436114887041024,1131227186,"March 10, 2020, 05:52:14 PM",THIS MAN IS A GENIUS he figured out the Corona...,179037,515867,1578,[]
3,5ea5fd064d254083e4e043dd,1239689136358985729,728371324184301568,"March 16, 2020, 11:04:57 PM",“corona time “😭😭😭😭 https://t.co/iXBMHVcFoY,132003,422685,1516,[]
4,5ea5feb64d254083e4e062bc,1242137430322446344,716432228,"March 23, 2020, 05:13:35 PM",This is Dr. Usama Riaz. He spent past weeks sc...,119601,331618,4876,[]


In [27]:
# using MongoDB 
# most popular original tweet ranked by the retweet_count, reply_count, and favorite_count 
pop_hottest = pd.DataFrame(tweet.find().sort([("retweet_count",-1),("reply_count",-1),("favorite_count",-1)]))
pop_hottest.head(5)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea5fccf4d254083e4e03e3f,1238264431320215553,1100261477989126145,"March 13, 2020, 12:43:40 AM",*corona virus enters my body*\n\nThe 4 Flintst...,237307,811062,1811,[]
1,5ea5fc9c4d254083e4e0378e,1240334979701395458,1112592502727548928,"March 18, 2020, 05:51:18 PM",When this Corona shit passes we have to promis...,181584,764405,1595,[]
2,5ea5fc604d254083e4e02c0c,1237436114887041024,1131227186,"March 10, 2020, 05:52:14 PM",THIS MAN IS A GENIUS he figured out the Corona...,179037,515867,1578,[]
3,5ea5fd064d254083e4e043dd,1239689136358985729,728371324184301568,"March 16, 2020, 11:04:57 PM",“corona time “😭😭😭😭 https://t.co/iXBMHVcFoY,132003,422685,1516,[]
4,5ea5feb64d254083e4e062bc,1242137430322446344,716432228,"March 23, 2020, 05:13:35 PM",This is Dr. Usama Riaz. He spent past weeks sc...,119601,331618,4876,[]


In [49]:
# using MongoDB 
# the most popular hashtag ranked by aggregating the lists of hashtags
from bson.son import SON
pipeline = [
     {"$unwind": "$hashtags"},
     {"$group": {"_id": "$hashtags", "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
] 
import pprint
pprint.pprint(list(db.tweet.aggregate(pipeline))[1:11])

[{'_id': 'corona', 'count': 437},
 {'_id': 'COVID19', 'count': 321},
 {'_id': 'coronavirus', 'count': 267},
 {'_id': 'Covid_19', 'count': 259},
 {'_id': 'lockdown', 'count': 129},
 {'_id': 'covid19', 'count': 115},
 {'_id': 'संतरामपालजी_का_सत्संग_सुनें', 'count': 114},
 {'_id': 'IndiaFightsCorona', 'count': 110},
 {'_id': 'Covid19', 'count': 106},
 {'_id': 'COVID', 'count': 89}]


Implement 2

In [51]:
# using SQL
# the most active verified user ranked by statuses_count
active_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status='True' ORDER BY statuses_count DESC")
active_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,47596019,Liputan6.com,liputan6dotcom,True,3687560,693,1381273,Jakarta Indonesia,7062
1,16973333,The Independent,Independent,True,3202840,1146,976895,"London, England",3
2,378809160,𝔾𝕀𝔻𝕀𝕋ℝ𝔸𝔽𝔽𝕀ℂ,Gidi_Traffic,True,1676895,5826,922265,Everywhere,14478
3,2475273985,Paytm Care,Paytmcare,True,248675,15,861933,India,402
4,15448383,Inquirer,inquirerdotnet,True,3076056,1852,858348,Makati City,34260


In [52]:
# using SQL
# the most popular verified user ranked by followers_count
pop_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status='True' ORDER BY followers_count DESC")
pop_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,18839785,Narendra Modi,narendramodi,True,55786179,2364,26559,India,0
1,807095,The New York Times,nytimes,True,46361159,904,393823,New York City,18483
2,145125358,Amitabh Bachchan,SrBachchan,True,41596464,1833,66428,"Mumbai, India",75
3,101311381,Shah Rukh Khan,iamsrk,True,40026738,77,58040,None,32
4,471741741,PMO India,PMOIndia,True,34461808,486,25996,India,0


In [53]:
# using SQL
# the most active location ranked by tweets count
pop_location = create_pandas_table("SELECT user_location, count(*) as amount FROM users_data WHERE user_location!='None' GROUP BY user_location ORDER BY amount DESC")
pop_location.head(5)

,user_location,amount
0,India,363
1,"New Delhi, India",185
2,United States,125
3,"Mumbai, India",121
4,"Lagos, Nigeria",112






Join the tweet table and user table

In [54]:
# using Pandas
full_table = pd.concat([tweet_table, user_table],axis=1)
full_table.shape
full_table.head(5)

# followings don't work
# full_table = tweet_table.join(user_table.set_index('user_id'), on='user_id')
# full_table = tweet_table.merge(user_table, on='user_id')
# full_table = tweet_table.append(user_table, ignore_index=True)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,5ea5fc384d254083e4e01ccb,1253949413191344128,207809313,"April 25, 2020, 07:30:12 AM",India’s war with Corona is ongoing.\n\nPlay yo...,398,2144,108,[FeedTheNeedy],375777294,TeéLaneeë🌺,TWD40_,False,637.0,408.0,18976.0,None,1325.0
1,5ea5fc384d254083e4e01ccc,1253992905703862272,858859031464751104,"April 25, 2020, 10:23:01 AM",VIDEO | 25.04.20\n\n&gt; Monsta X appears on Y...,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]",1132273796138905600,Terri Kamp,RampTheresa,False,1522.0,256.0,20861.0,None,29166.0
2,5ea5fc384d254083e4e01ccd,1253697983343816706,2719753171,"April 24, 2020, 02:51:06 PM","Sri ByReddy Rajasekhar Reddy garu,a leader,who...",2238,7243,47,[],301470336,BABA ALI,_FreshAA,False,9406.0,1870.0,264287.0,"Chicago, IL",857.0
3,5ea5fc384d254083e4e01cce,1254022773598572544,1091660129894838272,"April 25, 2020, 12:21:42 PM",@VinceMcMahon @TripleH We hereby honor to anno...,0,0,0,[],923169415054680064,Quantum,QuantumAspect,False,3196.0,1001.0,15219.0,The Netherlands,119803.0
4,5ea5fc384d254083e4e01ccf,1254022776094105602,375777294,"April 25, 2020, 12:21:43 PM",im making 17 a hr doing nothing 😭😭😭😭 i love th...,0,0,0,[],819200364675072000,Internet Historian,NetHistorian,False,259632.0,547.0,1087.0,None,4280.0


Implement 3

In [65]:
# using Pandas
# search the tweets that contain the keyword 'election'
election = full_table[full_table['text'].str.contains('election')]
len(election)

156

In [61]:
election['text'].head(5)

147     @JeffreyGuterman @realDonaldTrump @TripleH Tru...
706     @Babu_Bhaiyaa @realDonaldTrump @TripleH Ye cor...
845     @SueWhitaker11 The Tories are a totally FAKE '...
885     Nexttime anyone tells women to stop crying for...
1106    @dugarsanjays @AskAnshul Yahi to America maye ...
Name: text, dtype: object

In [69]:
# rank the tweets that contain the key word 'election' by favorite_count, retweet_count and reply_count
election.sort_values(by=['favorite_count','retweet_count','reply_count'],ascending=(False,False,False)).head(5)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
4985,5ea5fc754d254083e4e03044,1247124742638895105,2353605901,"April 06, 2020, 11:31:23 AM",The Democrats’ all-mail ballots proposal is a ...,5430,11868,1806,[],873428526359355392,Consciousness Rising,conciousness777,False,7420.0,1008.0,15223.0,"Miami, FL",15043.0
4235,5ea5fc664d254083e4e02d56,1254030161403674624,851211,"April 25, 2020, 12:51:03 PM",Milwaukee’s health commissioner has now tied 4...,1046,1204,76,[],463058325,Ilene Pretorius,Ilenetjie,False,354.0,344.0,9544.0,"Pretoria, South Africa",16563.0
4675,5ea5fc6e4d254083e4e02f0e,1253755113002369024,875856268056969216,"April 24, 2020, 06:38:07 PM","At this point, if you can’t see that Fake News...",312,988,25,[],1251310349669851138,Taylor Cox,RealTaylorCox,False,6.0,122.0,103.0,"Charleston, SC",51.0
4810,5ea5fc714d254083e4e02f95,1253766788522938368,1858288124,"April 24, 2020, 07:24:30 PM",Surely thinking you can inject yourself with h...,47,674,18,[],972682484075950082,ramandeep singh khanna,khannavaluerchd,False,43.0,268.0,154.0,Chandigarh,183.0
10293,5ea5fd134d254083e4e04500,1253411238580588547,11674792,"April 23, 2020, 07:51:41 PM",The elderly are being very unfair to Donald Tr...,112,607,14,[],1176866140427042817,Subhan Rind,rind_subhan,False,170.0,396.0,1462.0,None,4537.0


Implement 4

In [ ]:
# We can also link the results with the user table to find out the corresponding users 
# who tweet them and rank the results by the number of followers, number of friends, number of statuses, 
# number of favorites or combination of them.

In [ ]:
# using Pandas
# search for keyword 'election'


Implement 5

In [ ]:
# We also save the location information of users in the user table, even though it is self-reported and users 
# can choose whether they want to register or not. We still plan to set the related search in our application. 

# A user can search for the most preferred location ranked by amount of twitter users. 
# When a user searches a hashtag, the results can show the most active location related to the hashtag.

In [ ]:
# using Pandas
# search for keyword 'election'
